# Feature Engineering

In [1]:
pip install pandas numpy tldextract python-whois aiodns aiohttp idna joblib tqdm ipywidgets nest_asyncio brotli python-dateutil

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import math
import asyncio
import ssl
import socket
from datetime import datetime, timezone
from urllib.parse import urlparse, parse_qs
from concurrent.futures import ThreadPoolExecutor
from itertools import groupby
from dateutil import parser

import numpy as np
import pandas as pd
import tldextract
import idna
import whois
import aiodns
from joblib import Memory
import brotli

from tqdm import tqdm
from tqdm.asyncio import tqdm_asyncio as tqdma

import nest_asyncio
nest_asyncio.apply()


In [3]:
def lexical_features(df: pd.DataFrame) -> pd.DataFrame:
    u = df["url"]
    parsed = u.str.extract(
        r"^(?P<scheme>[^:]+)://(?P<rest>.+)$", expand=True).fillna("")
    scheme = parsed["scheme"]
    host_path = parsed["rest"]
    host = host_path.str.split("/", n=1).str[0]
    path_qr = host_path.str.split("/", n=1).str[1].fillna("")
    path = path_qr.str.split("?", n=1).str[0]
    query = path_qr.str.split("?", n=1).str[1].fillna("")
    frag = query.str.split("#", n=1).str[1].fillna("")
    query_only = query.str.split("#", n=1).str[0]
    v = pd.DataFrame(index=df.index)
    # Basic lengths
    v["url_length"]      = u.str.len().astype("int32")
    v["hostname_length"] = host.str.len().astype("int32")
    v["path_length"]     = path.str.len().astype("int32")
    v["query_length"]    = query_only.str.len().astype("int32")
    v["fragment_length"] = frag.str.len().astype("int32")
    # Symbol counts
    v["num_dots"]        = host.str.count(r"\.").astype("int16")
    v["num_hyphens"]     = u.str.count("-").astype("int16")
    v["num_underscores"] = u.str.count("_").astype("int16")
    v["digit_ratio"]     = (
        u.str.count(r"\d").astype("float32") / v["url_length"].clip(lower=1)
    )
    # Entropy
    def entropy(s):
        if not s: return 0.
        p = np.fromiter((s.count(c) / len(s) for c in set(s)), dtype=float)
        return -np.dot(p, np.log2(p))
    v["entropy_url"]  = u.map(entropy).astype("float32")
    v["entropy_host"] = host.map(entropy).astype("float32")
    # Vowel/consonant ratio
    vowels = host.str.count(r"[aeiouAEIOU]")
    conson = host.str.count(r"[A-Za-z]") - vowels
    v["vowel_cons_ratio"] = (vowels / conson.clip(lower=1)).astype("float32")
    # N-grams
    v["bigrams"]  = host.map(lambda s: len({s[i:i+2] for i in range(len(s)-1)})).astype("int16")
    v["trigrams"] = host.map(lambda s: len({s[i:i+3] for i in range(len(s)-2)})).astype("int16")
    # Phishing keywords and brand tokens (top 20+ from 2023-2025 literature)
    keywords = [
        "login","secure","update","verify","account","bank","signin","confirm",
        "apple","facebook","paypal","amazon","google","microsoft","netflix",
        "yahoo","dropbox","github","linkedin","ebay","wallet","invoice","payment",
        "support","alert","service","admin","webscr","session","billing"
    ]
    for kw in keywords:
        v[f"kw_{kw}_count"] = u.str.lower().str.count(kw).astype("int8")
    # Suspicious patterns
    v["has_at_symbol"]   = u.str.contains("@").astype("int8")
    v["has_double_slash"] = u.str.slice(8).str.contains("//").astype("int8")
    v["has_ip_address"]  = host.str.match(r"^(?:\d{1,3}\.){3}\d{1,3}$").astype("int8")
    v["uses_https"]      = (scheme == "https").astype("int8")
    v["param_count"]     = query_only.str.count("&").add(query_only.ne("").astype(int)).astype("int16")
    v["subdomain_count"] = host.fillna("").str.split(".").map(lambda x: max(len(x)-2, 0)).astype("int16")
    # Uppercase ratio
    v["uppercase_ratio"] = (
        u.str.count(r"[A-Z]").astype("float32") / v["url_length"].clip(lower=1)
    )
    # Char repetition (max run of same char class)
    v["max_char_run"] = u.map(lambda s: max([len(list(g)) for _, g in groupby(s)], default=0)).astype("int16")
    # Kolmogorov complexity (compressed size/len, optional)
    try:
        import brotli
        v["kolmogorov"] = u.map(lambda s: len(brotli.compress(s.encode(), quality=4)) / max(len(s),1)).astype("float32")
    except ImportError:
        v["kolmogorov"] = 0.0
    # TLD feature (optional: can be filled with .5 if not available)
    tld = host.map(lambda h: tldextract.extract(h).suffix)
    tld_legit = {"com":0.8,"org":0.7,"net":0.7,"ru":0.2,"cn":0.2,"xyz":0.1,"top":0.1,"info":0.3,"site":0.2,"online":0.2}
    v["tld_legit_prob"] = tld.map(lambda s: tld_legit.get(s,0.5)).astype("float32")
    return v


In [4]:
CACHE = Memory(".cache_feat", verbose=0)
DNS_RESOLVER = aiodns.DNSResolver(timeout=6)
SSL_CTX = ssl.create_default_context()

def _whois_dates(domain: str):
    try:
        w = whois.whois(domain)
        cd, ed = w.creation_date, w.expiration_date
        if isinstance(cd, list): cd = cd[0]
        if isinstance(ed, list): ed = ed[0]
        # Parse to datetime if string
        if isinstance(cd, str):
            cd = parser.parse(cd)
        if isinstance(ed, str):
            ed = parser.parse(ed)
        # Make both timezone-aware in UTC
        if cd is not None:
            if cd.tzinfo is None or cd.tzinfo.utcoffset(cd) is None:
                cd = cd.replace(tzinfo=timezone.utc)
            else:
                cd = cd.astimezone(timezone.utc)
        if ed is not None:
            if ed.tzinfo is None or ed.tzinfo.utcoffset(ed) is None:
                ed = ed.replace(tzinfo=timezone.utc)
            else:
                ed = ed.astimezone(timezone.utc)
        return cd, ed
    except Exception:
        return None, None

async def dns_features(host: str) -> dict:
    out = {"a_count":0,"mx_count":0,"ns_count":0,"reverse_dns":0}
    try:
        a = await DNS_RESOLVER.query(host, "A")
        out["a_count"] = len(a)
        try:
            rev = ".".join(reversed(host.split("."))) + ".in-addr.arpa"
            await DNS_RESOLVER.query(rev, "PTR")
            out["reverse_dns"]=1
        except: pass
    except: pass
    for rec,tp in (("MX","mx_count"),("NS","ns_count")):
        try:
            ans = await DNS_RESOLVER.query(host, rec)
            out[tp] = len(ans)
        except: pass
    return out

def _ssl_worker(host: str):
    res = {"cert_valid_days":0,"issuer_len":0,"days_since_issue":0,"self_signed":0}
    try:
        with SSL_CTX.wrap_socket(socket.socket(), server_hostname=host) as s:
            s.settimeout(8)
            s.connect((host,443))
            cert = s.getpeercert()
            nb = datetime.strptime(cert["notBefore"],"%b %d %H:%M:%S %Y %Z")
            na = datetime.strptime(cert["notAfter"], "%b %d %H:%M:%S %Y %Z")
            res["cert_valid_days"] = (na-nb).days
            res["days_since_issue"]= (datetime.now()-nb).days
            iss = dict(x[0] for x in cert.get("issuer", []))
            res["issuer_len"] = len(iss.get("commonName",""))
            res["self_signed"] = int(iss.get("organizationalUnitName","")=="Self-signed")
    except: pass
    return res

async def gather_net_features(hosts: list[str]) -> pd.DataFrame:
    loop  = asyncio.get_running_loop()
    pool  = ThreadPoolExecutor(max_workers=8)
    dns_coroutines  = [dns_features(h) for h in hosts]
    ssl_futures     = [loop.run_in_executor(pool, _ssl_worker, h) for h in hosts]
    dns_out = []
    for coro in tqdma.as_completed(dns_coroutines, total=len(hosts), desc="Async DNS"):
        dns_out.append(await coro)
    ssl_out = []
    for fut in tqdma.as_completed(ssl_futures, total=len(hosts), desc="TLS handshake"):
        ssl_out.append(await fut)
    whois_dates = []
    with ThreadPoolExecutor(max_workers=8) as wpool:
        for fut in tqdma.as_completed(
                [loop.run_in_executor(wpool, _whois_dates, tldextract.extract(h).registered_domain)
                 for h in hosts],
                total=len(hosts), desc="WHOIS"):
            whois_dates.append(await fut)
    today = datetime.now(timezone.utc)
    rows = []
    for h, dns_d, ssl_d, (cd, ed) in zip(hosts, dns_out, ssl_out, whois_dates):
        age = (today-cd).days if cd else 0
        to_exp = (ed-today).days if ed else 0
        rows.append(dict(
            host=h,
            domain_age_days=age,
            days_to_expiry=to_exp,
            **dns_d, **ssl_d
        ))
    return pd.DataFrame(rows).set_index("host")


In [5]:
def batch_extract_notebook(urls: list[str]) -> pd.DataFrame:
    return asyncio.get_event_loop().run_until_complete(async_extract(urls))

async def async_extract(urls: list[str]) -> pd.DataFrame:
    df = pd.DataFrame({"url": urls})
    with tqdm(total=len(df), desc="Lexical features", unit="url") as pb:
        feats = lexical_features(df)
        pb.update(len(df))
    hosts = df["url"].map(lambda u: urlparse(u).hostname or "")
    unique_hosts = hosts.unique().tolist()
    net_df = await gather_net_features(unique_hosts)
    feats = feats.join(net_df, on=hosts)
    feats.insert(0, "url", urls)
    return feats


In [ ]:
DATA_PATH = r"C:\prateek\projects\linkload\backend\ml_models\phishing_detection\data\labeled_url_dataset.csv"
OUTPUT_PATH = r"C:\prateek\projects\linkload\backend\ml_models\phishing_detection\data\phishing_dataset_with_features.csv"
BATCH_SIZE = 1000   # Adjust based on your resources

reader = pd.read_csv(DATA_PATH, chunksize=BATCH_SIZE)
first = True
for chunk in tqdm(reader, desc="Overall Progress"):
    chunk = chunk.dropna(subset=['url', 'label']).reset_index(drop=True)
    urls = chunk['url'].tolist()
    labels = chunk['label'].tolist()
    feat_df = batch_extract_notebook(urls)
    feat_df['label'] = labels
    # Save batch to CSV (append mode)
    if first:
        feat_df.to_csv(OUTPUT_PATH, index=False, mode='w')
        first = False
    else:
        feat_df.to_csv(OUTPUT_PATH, index=False, mode='a', header=False)
    print(f"Processed {len(urls)} URLs, saved to {OUTPUT_PATH}")


TLS handshake: 100%|██████████| 816/816 [01:12<00:00, 11.18it/s]
C:\Users\sendt\AppData\Local\Temp\ipykernel_30400\1173181103.py:80: DeprecationWarning: The 'registered_domain' property is deprecated and will be removed in the next major version. Use 'top_domain_under_public_suffix' instead, which has the same behavior but a more accurate name.
  [loop.run_in_executor(wpool, _whois_dates, tldextract.extract(h).registered_domain)
2025-07-09 20:52:02,316 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed

2025-07-09 20:52:03,611 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed2025-07-09 20:52:06,220 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [WinError 10061] No connection could be made because the target machine actively refused it
2025-07-09 20:52:14,703 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11

Processed 1000 URLs, saved to C:\prateek\projects\linkload\backend\ml_models\phishing_detection\data\phishing_dataset_with_features.csv


TLS handshake: 100%|██████████| 832/832 [01:30<00:00,  9.20it/s]
C:\Users\sendt\AppData\Local\Temp\ipykernel_30400\1173181103.py:80: DeprecationWarning: The 'registered_domain' property is deprecated and will be removed in the next major version. Use 'top_domain_under_public_suffix' instead, which has the same behavior but a more accurate name.
  [loop.run_in_executor(wpool, _whois_dates, tldextract.extract(h).registered_domain)
2025-07-09 21:02:41,680 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed

2025-07-09 21:02:41,685 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
2025-07-09 21:02:42,375 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed2025-07-09 21:02:42,421 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
2025-07-09 21:02:45,687 - whois.who

Processed 1000 URLs, saved to C:\prateek\projects\linkload\backend\ml_models\phishing_detection\data\phishing_dataset_with_features.csv


TLS handshake: 100%|██████████| 803/803 [01:09<00:00, 11.60it/s]
C:\Users\sendt\AppData\Local\Temp\ipykernel_30400\1173181103.py:80: DeprecationWarning: The 'registered_domain' property is deprecated and will be removed in the next major version. Use 'top_domain_under_public_suffix' instead, which has the same behavior but a more accurate name.
  [loop.run_in_executor(wpool, _whois_dates, tldextract.extract(h).registered_domain)
2025-07-09 21:11:28,056 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
2025-07-09 21:11:30,365 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed

2025-07-09 21:11:32,064 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed2025-07-09 21:11:32,806 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [WinError 10061] No connection could be made because the target mach

...

TLS handshake: 100%|██████████| 803/803 [01:19<00:00, 10.16it/s]
C:\Users\sendt\AppData\Local\Temp\ipykernel_30400\1173181103.py:80: DeprecationWarning: The 'registered_domain' property is deprecated and will be removed in the next major version. Use 'top_domain_under_public_suffix' instead, which has the same behavior but a more accurate name.
  [loop.run_in_executor(wpool, _whois_dates, tldextract.extract(h).registered_domain)
2025-07-10 14:08:03,707 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
2025-07-10 14:08:04,356 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [WinError 10061] No connection could be made because the target machine actively refused it

2025-07-10 14:08:04,363 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed2025-07-10 14:08:11,170 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11

Processed 1000 URLs, saved to C:\prateek\projects\linkload\backend\ml_models\phishing_detection\data\phishing_dataset_with_features.csv


TLS handshake: 100%|██████████| 802/802 [01:23<00:00,  9.55it/s]
C:\Users\sendt\AppData\Local\Temp\ipykernel_30400\1173181103.py:80: DeprecationWarning: The 'registered_domain' property is deprecated and will be removed in the next major version. Use 'top_domain_under_public_suffix' instead, which has the same behavior but a more accurate name.
  [loop.run_in_executor(wpool, _whois_dates, tldextract.extract(h).registered_domain)
2025-07-10 14:13:27,214 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [WinError 10061] No connection could be made because the target machine actively refused it
2025-07-10 14:13:32,735 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed

2025-07-10 14:13:34,931 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
2025-07-10 14:13:35,887 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 1

Processed 1000 URLs, saved to C:\prateek\projects\linkload\backend\ml_models\phishing_detection\data\phishing_dataset_with_features.csv


Lexical features: 100%|██████████| 1000/1000 [00:00<00:00, 3106.45url/s]
